# Imports y configuraciones iniciales

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from quickdraw import QuickDrawDataGroup

from bwb.distributions import DistributionDraw

Importamos los datos utilizando el archivo npy que proporciona Google Cloud.

In [2]:
data = np.load(r"..\data\face.npy")
data.shape

(161666, 784)

Hacemos una lista de caritas, fijando un generador con la semilla 42 para tener resultados replicables.

In [3]:
rng = np.random.default_rng(42)
faces: list[DistributionDraw] = list()
n_faces = len(data)

for i in range(n_faces):
    arr = 255 - data[i, :].reshape(28, 28)
    dd_arr = DistributionDraw.fromarray(arr, seed=rng)
    faces.append(dd_arr)

Observamos la primera carita

In [4]:
faces[0].image

Muestrear con respecto a una carita (la primera para simplificar)

In [5]:
n = 3
x = faces[0].rvs(size=n)
x

[(21, 15), (11, 10), (22, 18)]

Defino
\begin{equation}
\tilde\Lambda_n(m) = \prod_{i=1}^{n} f_m(x_i)
\end{equation}

A través de la función `lambda_n_tilde`

In [6]:
def lambda_n_tilde(mu: DistributionDraw, data: list[tuple[int, int]]):
    evaluations = []
    for i in range(len(data)):
        evaluations.append(mu.matrix[data[i]])
    return np.prod(evaluations)
lambda_n_tilde(mu=faces[0], data=x)

6.001854530919468e-08

# Metropolis Algorithm

En esta sección se implementa el *Metropolis Algorithm*. Este posee la siguiente matriz $R$:
\begin{equation}
R_{\mu, \nu} = (\text{degr}(\mu))^{-1} = \frac{1}{|\mathcal{M} - 1|}
\end{equation}
donde lo último es por la forma que posee el grafo escogido.

In [7]:
%%time
EPS = 1e-16  # Holgura para el determinador, para evitar dividir por 0

N = 10_000  # 10_000
mu = []

# Paso 1. Empezar con una distribución inicial mu^(0) \in M
current_face = int(rng.choice(n_faces))
mu.append(faces[current_face])  # mu^(0)

# Paso 2.
for i in range(N):
    # Paso a. muestrear u ~ U[0, 1]
    u = rng.uniform(low=0, high=1)

    # Paso b. muestrear mu^star de acuerdo a R partiendo desde mu^(i)
    possible_faces = list(range(n_faces))
    possible_faces.remove(current_face)
    next_face = int(rng.choice(possible_faces))
    mu_star = faces[next_face]

    # Paso c.
    # Empecemos calculando R
    R_mu_star_mu_i = 1 / (n_faces - 1)
    R_mu_i_mu_star = 1 / (n_faces - 1)
    # Luego calculamos la probabilidad de aceptación
    A_mu_i_mu_star = min(1, (lambda_n_tilde(mu_star, data=x) * R_mu_star_mu_i)
                            / (lambda_n_tilde(mu[i], data=x) * R_mu_i_mu_star + EPS))

    if u < A_mu_i_mu_star:
        mu.append(mu_star)

        print(f"{i = }")
        print(f"{A_mu_i_mu_star = }")
        print(f"{u = }")

    else:
        mu.append(mu[i])

i = 1
A_mu_i_mu_star = 1
u = 0.9756223516367559
i = 36
A_mu_i_mu_star = 0.6077508474610039
u = 0.13979699812765745
i = 86
A_mu_i_mu_star = 1
u = 0.34686980453483707
i = 90
A_mu_i_mu_star = 0.8965082731291945
u = 0.08272999992243857
i = 99
A_mu_i_mu_star = 1
u = 0.14024908899861077
i = 163
A_mu_i_mu_star = 0.5835861198245356
u = 0.3741840434071827
i = 168
A_mu_i_mu_star = 1
u = 0.8165687634239104
i = 174
A_mu_i_mu_star = 1
u = 0.16553172273527816
i = 193
A_mu_i_mu_star = 0.19510215262041755
u = 0.0991131415798755
i = 197
A_mu_i_mu_star = 1
u = 0.6424832759955477
i = 199
A_mu_i_mu_star = 1
u = 0.5360680359476373
i = 223
A_mu_i_mu_star = 0.09514292763359644
u = 0.08449321882352023
i = 224
A_mu_i_mu_star = 1
u = 0.13740793003780216
i = 229
A_mu_i_mu_star = 1
u = 0.2533833538391872
i = 238
A_mu_i_mu_star = 0.15070688454300812
u = 0.11973358850112281
i = 251
A_mu_i_mu_star = 1
u = 0.5529930652528999
i = 265
A_mu_i_mu_star = 0.9480384683181111
u = 0.9184418955155542
i = 278
A_mu_i_mu_star = 0

Empecemos observando las distintas imágenes que generó

In [8]:
# Primera imagen generada
mu[0].image

In [9]:
# Última imagen generada
mu[-1].image

# Gibbs Sampler

Ahora empecemos a generar la otra variación del algoritmo: *Gibbs Sampler*. Después de unos cálculos, este posee la siguiente matriz $R$:
\begin{equation}
R_{\mu, \nu} = \frac{
\tilde \Lambda_n(\nu)
}{
\sum_{\tilde \nu \in \mathcal{M}} \tilde \Lambda_n(\tilde \nu) - \tilde \Lambda_n(\mu)
}
\end{equation}
con la función $\tilde\Lambda_n$ definida como antes.

Notemos que, la gran diferencia con el otro algoritmo es la forma de calcular la matriz $R$. Por este motivo se copiará y pegará el algoritmo anterior, modificando este valor.

In [10]:
%%time
# Cálculo de la sumatoria, que es un valor genérico para todos
sum_lambdas_n = sum([lambda_n_tilde(nu_tilde, data=x) for nu_tilde in faces])
sum_lambdas_n

CPU times: total: 8.08 s
Wall time: 8.12 s


0.0012105611403291847

In [11]:
%%time
EPS = 1e-16  # Holgura para el determinador, para evitar dividir por 0

N = 10_000
mu = []

# Paso 1. Empezar con una distribución inicial mu^(0) \in M
current_face = int(rng.choice(n_faces))
mu.append(faces[current_face])  # mu^(0)

# Paso 2.
for i in range(N):
    # Paso a. muestrear u ~ U[0, 1]
    u = rng.uniform(low=0, high=1)

    # Paso b. muestrear mu^star de acuerdo a R partiendo desde mu^(i)
    possible_faces = list(range(n_faces))
    possible_faces.remove(current_face)
    next_face = int(rng.choice(possible_faces))
    mu_star = faces[next_face]

    # Paso c.
    # Calculo cache, dado que se ocupar varias veces
    lambda_n_tilde_mu_star = lambda_n_tilde(mu_star, data=x)
    lambda_n_tilde_mu_i = lambda_n_tilde(mu[i], data=x)
    # Cálculo de las transiciones R
    R_mu_star_mu_i = (lambda_n_tilde_mu_star
                      / (sum_lambdas_n - lambda_n_tilde_mu_i))
    R_mu_i_mu_star = (lambda_n_tilde_mu_i
                      / (sum_lambdas_n - lambda_n_tilde_mu_star))
    # Luego calculamos la probabilidad de aceptación
    A_mu_i_mu_star = min(1, (lambda_n_tilde_mu_star * R_mu_star_mu_i)
                         / (lambda_n_tilde_mu_i * R_mu_i_mu_star + EPS))

    if u < A_mu_i_mu_star:
        mu.append(mu_star)

        print(f"{i = }")
        print(f"{A_mu_i_mu_star = }")
        print(f"{u = }")

    else:
        mu.append(mu[i])

i = 0
A_mu_i_mu_star = 1
u = 0.49347953301914227
i = 16
A_mu_i_mu_star = 1
u = 0.6607049978861596
i = 152
A_mu_i_mu_star = 1
u = 0.2601446921997975
i = 273
A_mu_i_mu_star = 0.48946934975523215
u = 0.24281917477268689
i = 292
A_mu_i_mu_star = 1
u = 0.7002714728296972
i = 313
A_mu_i_mu_star = 1
u = 0.5245028101364752
i = 316
A_mu_i_mu_star = 0.6407359983808782
u = 0.11048622811819742
i = 409
A_mu_i_mu_star = 0.2670068171324143
u = 0.2086953672495787
i = 433
A_mu_i_mu_star = 1
u = 0.03392140053601933
i = 478
A_mu_i_mu_star = 0.20191681640586567
u = 0.017722379285222356
i = 484
A_mu_i_mu_star = 0.7354626282948157
u = 0.416737650585681
i = 491
A_mu_i_mu_star = 0.7986692788914644
u = 0.5365138570536546
i = 497
A_mu_i_mu_star = 1
u = 0.051134561745864504
i = 499
A_mu_i_mu_star = 0.900184367923984
u = 0.5387891305470927
i = 502
A_mu_i_mu_star = 0.5928794554051244
u = 0.06350078724999064
i = 513
A_mu_i_mu_star = 0.6885623608830372
u = 0.4579650576825216
i = 519
A_mu_i_mu_star = 0.43783634301367

Ideas para continuar:
* Dejar guardadas de alguna forma las simulaciones realizadas
* Intentar hacer el mismo experimento, pero haciendo que el soporte de las funciones sea distinto de 0 (es decir, que todos tengan el mismo soporte), para que la función $\tilde \Lambda_n$ no se anule, y no posea una probabilidad de aceptación nula